In [ ]:
# IMPORTS

import os 
import time
from io import StringIO

import re

import numpy as np
import pandas as pd

import openpyxl

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from numpy.polynomial.polynomial import Polynomial
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error


import jou_gen as jg

import ansys.fluent.core as pyfluent

In [ ]:
fp_temp = r'D:\ANSYS Fluent Projects\temp'
folder_path = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\19MPE_porous\fluent\C1_100Lh\v2'

In [ ]:
jg.convert_report(folder_path,'report_mdot','mdot',folder_path,output_file_name_wo_extension='report_mdot')
jg.convert_report(folder_path,'report_sp','sp',folder_path,output_file_name_wo_extension='report_sp')

In [ ]:
df = pd.read_csv(os.path.join(folder_path,'report_mdot.csv'))
df2 = pd.read_csv(os.path.join(folder_path,'report_sp.csv'))

In [ ]:
dfs = df[df['Component'].str.contains('porous')]
dfs = dfs[~dfs['Component'].str.contains(':1')]
dfs.reset_index(drop=True,inplace=True)

dfs2 = df2[df2['Component'].str.contains('porous')]
dfs2 = dfs2[~dfs2['Component'].str.contains(':1')]
dfs2.reset_index(drop=True,inplace=True)

In [ ]:
num_MPE_list = list(np.concatenate([np.arange(12,19+1),np.arange(110,120+1)]))
new_num_MPE_list = [1,19]
num_ch = list(np.arange(1,16+1))

In [ ]:
coeff = 1411785.

In [ ]:
def extract_values(input_str):
    # Define a regular expression pattern to match the desired format
    pattern = r'rep-mpe(\d+)-chporous(\d+)-rep-mpe(\d+)-(\w+)'
    
    # Use the re.match() function to search for the pattern in the input string
    match = re.match(pattern, input_str)
    
    if match:
        # Extract the values from the matched groups
        num_MPE = match.group(1)
        num_ch = match.group(2)
        zone = match.group(4)
        
        # Return the extracted values
        return num_MPE_list.index(int(num_MPE))+1, int(num_ch), zone
    else:
        # Return None if the pattern is not found in the input string
        return None
    
def rename_component(component):
    num_MPE, num_ch, zone = extract_values(component)
    new_name = f'MPE{num_MPE}_ch{num_ch}_{zone}'
    return new_name

In [ ]:
def pr(dfs):
    
    dfs['Component'] = dfs['Component'].apply(rename_component)
    dfs['MPE'] = dfs['Component'].apply(lambda x: int(x.split('_')[0].split('MPE')[1]))
    dfs['ch'] = dfs['Component'].apply(lambda x: int(x.split('_')[1].split('ch')[1]))
    dfs['zone'] = dfs['Component'].apply(lambda x: x.split('_')[2])

    dfs.sort_values(by=['MPE','ch'],inplace=True)
    dfs.reset_index(drop=True,inplace=True)

In [ ]:
pr(dfs)
dfs['mdot'] = dfs['mdot'].apply(lambda x: abs(x))
pr(dfs2)

In [ ]:
dfs

In [ ]:
dfs2

In [ ]:
dfsD = dfs[dfs['zone'] != 'distrib']
dfsD.reset_index(drop=True,inplace=True)
dfsD['mdot_uniform'] = dfsD['mdot'].sum()/dfsD.shape[0]

In [ ]:
# Assuming dfsD is your DataFrame and you want to plot 'mdot' against its index
scatter_plot = go.Scatter(x=dfsD.index, y=dfsD['mdot']/dfsD['mdot_uniform'], mode='markers')
# scatter_uniform = go.Scatter(x=dfsD.index, y=dfsD['mdot_uniform'], mode='lines', name='Mean')

# Create the layout (if needed)
layout = go.Layout(title='Scatter Plot', xaxis=dict(title='Index'), yaxis=dict(title='mdot'))

# Create the figure
fig = go.Figure(data=[scatter_plot
                    #   ,scatter_uniform
                      ],
                      layout=layout)

# Show the plot
fig.show()

In [ ]:
# Filter rows with zone 'distrib'
dfs2_distrib = dfs2[dfs2['zone'] == 'distrib']

# Filter rows with zone 'coll'
dfs2_coll = dfs2[dfs2['zone'] == 'coll']

# Merge the two dataframes on MPE and ch columns
merged_df = pd.merge(dfs2_distrib, dfs2_coll, on=['MPE', 'ch'], suffixes=('_distrib', '_coll'))

# Calculate the difference of 'sp' values
merged_df['sp_difference'] = merged_df['sp_distrib'] - merged_df['sp_coll']

# Select the desired columns
result_df = merged_df[['MPE', 'ch', 'sp_difference']]

# Display the resulting dataframe
result_df

In [ ]:
result_df['DP_wanted'] = dfsD['mdot'] * coeff

In [ ]:
result_df['diff in percentage'] = (result_df['sp_difference'] - result_df['DP_wanted']) / result_df['DP_wanted']

In [ ]:
result_df

In [ ]:
result_df['diff in percentage'].describe()